## Importando as bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.11.0'

# Parte 1 - Pré-processamento de dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importando o dataset

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Análise de Churn com Artificial Neural Networks/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X) 

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


## Codificando os dados categóricos

Codificando a coluna 'Gender'

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding a coluna 'Geography'

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


## Dividindo o dataset em treino e tese

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling (essencial para deep learning)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Parte 2 - Construindo a ANN

## Iniciando o ANN

In [ ]:
ann = tf.keras.models.Sequential()

## Adicionando imput layer e o primeiro hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Adicionando o segundo hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Adicionando o output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


#Como se trata de um resultado binário (que pode ser respondido em uma única
#coluna), só precisa ter 1 unidade. Caso fosse por mais, 3 por exemplo, seriam
#necessárias 3 unidades, pois seria equivalente a quantidade de colunas da
#resposta (0 0 1, 0 1 0, etc).

# Parte 3 - Treinando a ANN

## Compilando a ANN

In [ ]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#se não fosse binário, o loss seria "categorical_crossentropy"

## Treinando a ANN com o Training Set

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)
#o número de epochs não pode ser muito baixo, de forma que o algoritmo aprenda
#bem

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6788 - accuracy: 0.6081
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6013 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5479 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5105 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4881 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4756 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4680 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4627 - accuracy: 0.7960
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4584 - accuracy: 0.7960
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4547 - accura

# Parte 4 - Fazendo as previsões e avaliando o modelo

## Prevendo o resultado de uma única observação

**Dever de Casa**
Use nosso modelo ANN para prever se o cliente com as seguintes informações vai deixar o banco:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

Então, daremos adeus ao nosso cliente?

In [ ]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

1/1 [==============================] - 0s 25ms/step
[[False]]


## Prevendo o resultado do test set

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 950us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Fazendo a matriz de confusão

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1555   40]
 [ 372   33]]


0.794